# LIBRARY LOADS

In [1]:
import sys
sys.executable

'/beegfs/home/r.zagidullin/RAPIDS_VERSIONS/LightAutoML_GPU/lama_venv/bin/python3'

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy import sparse

from lightautoml.reader.cudf_reader import CudfReader
from lightautoml.reader.hybrid_reader import HybridReader
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task

import cudf
import dask_cudf

from numba import cuda
import cupy as cp

from lightautoml.transformers.base import SequentialTransformer, UnionTransformer


from lightautoml.transformers import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml.transformers import numeric, categorical, datetime

from lightautoml.pipelines.utils import get_columns_by_role

from dask.distributed import Client
from dask_cuda import LocalCUDACluster

import os

In [3]:
from lightautoml.reader.daskcudf_reader import DaskCudfReader

In [4]:
cluster = LocalCUDACluster(rmm_managed_memory=True,
                           CUDA_VISIBLE_DEVICES="0",
                           protocol="ucx", enable_nvlink=True,
                           memory_limit="8GB")

client = Client(cluster)
client.run(cudf.set_allocator, "managed")
#client.run(os.getpid)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize


{'ucx://127.0.0.1:45491': None}

# DATA AND READERS PREPARATION

In [5]:
data = pd.read_csv('./application_train.csv')

In [6]:
data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                   ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data.drop(['DAYS_BIRTH',  'DAYS_EMPLOYED'], axis = 1, inplace = True)


In [7]:
task = task = Task('binary')
adv_roles = True
parts = 1
reader = PandasToPandasReader(task, advanced_roles=adv_roles,
                              n_jobs=1)
gpu_reader = CudfReader(task, advanced_roles=adv_roles, n_jobs=1)
dd_reader = DaskCudfReader(task, advanced_roles=adv_roles, 
                           n_jobs=1, index_ok = True, compute=False, npartitions=parts)

hy_reader = HybridReader(task, num_cpu_readers=1,
                         num_gpu_readers=1, gpu_ratio=0.5,
                         output='mgpu', advanced_roles=adv_roles, index_ok = True,
                         compute = False,
                         npartitions=parts, n_jobs=1)

In [8]:
gpu_data = cudf.DataFrame.from_pandas(data, nan_as_null=False)
dd_data = dask_cudf.from_cudf(gpu_data, npartitions=parts)

# TIMING READERS

In [9]:
%%time
ds = reader.fit_read(data, roles = {'target': 'TARGET'})

CPU times: user 23.7 s, sys: 784 ms, total: 24.4 s
Wall time: 23.9 s


In [10]:
%%time
gpu_ds = gpu_reader.fit_read(gpu_data, roles = {'target': 'TARGET'})

CPU times: user 13 s, sys: 1.93 s, total: 14.9 s
Wall time: 15.8 s


In [11]:
%%time
dd_ds = dd_reader.fit_read(dd_data, roles = {'target': 'TARGET'})

8.649658411741257e-07 nothing
0.00013847299851477146 valid_roels
4.66126948595047e-07 index_ok
2.2132182493805885e-05 initialize
0.04904052382335067 set-data
CPU times: user 19.3 s, sys: 4.08 s, total: 23.4 s
Wall time: 41.3 s


In [12]:
%%time
dd_ds = hy_reader.fit_read(data, roles={'target': 'TARGET'})

35.64038692810573 hybrid finished like
1.1099036782979965e-06 nothing
0.0001772991381585598 valid_roels
6.060581654310226e-07 index_ok
4.237005487084389e-05 initialize
0.05186896398663521 set-data
6.368398246122524 final fit_read
CPU times: user 6.47 s, sys: 2.39 s, total: 8.86 s
Wall time: 42 s


# TIMING LABEL ENCODER

In [ ]:
trf = categorical.LabelEncoder()
gpu_trf = categorical_gpu.LabelEncoder_gpu()
dd_trf = categorical_gpu.LabelEncoder_gpu()

cats = ds[:, get_columns_by_role(ds, 'Category')]
gpu_cats = gpu_ds[:, get_columns_by_role(gpu_ds, 'Category')]
dd_cats = dd_ds[:, get_columns_by_role(dd_ds, 'Category')]

print(cats.shape, gpu_cats.shape, dd_cats.shape)

In [ ]:
%%timeit
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

# TIMING TARGET ENCODER

In [ ]:
trf = SequentialTransformer(
    [categorical.LabelEncoder(), categorical.TargetEncoder()]
)
gpu_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.TargetEncoder_gpu()]
)
dd_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.TargetEncoder_gpu()]
)


In [ ]:
%%timeit
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

# TIMING OTHER CATEGORICAL TRANSFORMERS

## FreqEncoder

In [ ]:
trf = categorical.FreqEncoder()
gpu_trf = categorical_gpu.FreqEncoder_gpu()
dd_trf = categorical_gpu.FreqEncoder_gpu()

In [ ]:
%%timeit
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

## OrdinalEncoder

In [ ]:
trf = categorical.OrdinalEncoder()
gpu_trf = categorical_gpu.OrdinalEncoder_gpu()
dd_trf = categorical_gpu.OrdinalEncoder_gpu()

In [ ]:
%%timeit
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

## OHEEncoder

In [ ]:
trf = SequentialTransformer(
    [categorical.LabelEncoder(), categorical.OHEEncoder(make_sparse=True)]
)
gpu_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.OHEEncoder_gpu(make_sparse=False)]
)
dd_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.OHEEncoder_gpu(make_sparse=False)]
)

In [ ]:
%%timeit
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

## CatIntersections

In [ ]:
trf = SequentialTransformer(
    [categorical.LabelEncoder(), categorical.CatIntersectstions()]
)
gpu_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.CatIntersections_gpu()]
)
dd_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.CatIntersections_gpu()]
)

In [ ]:
%%time
enc = trf.fit_transform(cats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)

# TIMING DATETIME TRANSFORMERS

## TimeToNum

In [ ]:
trf = datetime.TimeToNum()
gpu_trf = datetime_gpu.TimeToNum_gpu()
dd_trf = datetime_gpu.TimeToNum_gpu()

dats = ds[:, get_columns_by_role(ds, 'Datetime')]
gpu_dats = gpu_ds[:, get_columns_by_role(gpu_ds, 'Datetime')]
dd_dats = dd_ds[:, get_columns_by_role(dd_ds, 'Datetime')]

print(dats.shape, gpu_dats.shape, dd_dats.shape)

In [ ]:
%%time
enc = trf.fit_transform(dats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_dats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_dats)

## BaseDiff

In [ ]:
trf = datetime.BaseDiff(base_names=[dats.features[0]], diff_names=[dats.features[1]])
gpu_trf = datetime_gpu.BaseDiff_gpu(base_names=[dats.features[0]], diff_names=[dats.features[1]])
dd_trf = datetime_gpu.BaseDiff_gpu(base_names=[dats.features[0]], diff_names=[dats.features[1]])

In [ ]:
%%time
enc = trf.fit_transform(dats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_dats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_dats)

## DateSeasons

In [ ]:
trf = datetime.DateSeasons()
gpu_trf = datetime_gpu.DateSeasons_gpu()
dd_trf = datetime_gpu.DateSeasons_gpu()

In [ ]:
%%time
enc = trf.fit_transform(dats)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_dats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_dats)

# TIMING NUMERICAL TRANSFORMERS

## NaNFlags

In [ ]:
trf = numeric.NaNFlags()
gpu_trf = numeric_gpu.NaNFlags_gpu()
dd_trf = numeric_gpu.NaNFlags_gpu()

nums = ds[:, get_columns_by_role(ds, 'Numeric')]
gpu_nums = gpu_ds[:, get_columns_by_role(gpu_ds, 'Numeric')]
dd_nums = dd_ds[:, get_columns_by_role(dd_ds, 'Numeric')]

print(nums.shape, gpu_nums.shape, dd_nums.shape)

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_nums)

## FillnaMedian

In [ ]:
trf = numeric.FillnaMedian()
gpu_trf = numeric_gpu.FillnaMedian_gpu()
dd_trf = numeric_gpu.FillnaMedian_gpu()

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_nums)

## FillInf

In [ ]:
trf = numeric.FillInf()
gpu_trf = numeric_gpu.FillInf_gpu()
dd_trf = numeric_gpu.FillInf_gpu()

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_nums)

## LogOdds

In [ ]:
trf = numeric.LogOdds()
gpu_trf = numeric_gpu.LogOdds_gpu()
dd_trf = numeric_gpu.LogOdds_gpu()

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_nums)

## StandardScaler

In [ ]:
trf = numeric.StandardScaler()
gpu_trf = numeric_gpu.StandardScaler_gpu()
dd_trf = numeric_gpu.StandardScaler_gpu()

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_nums)

## QuantileBinning

In [ ]:
trf = numeric.QuantileBinning()
gpu_trf = numeric_gpu.QuantileBinning_gpu()
dd_trf = numeric_gpu.QuantileBinning_gpu()

In [ ]:
%%time
enc = trf.fit_transform(nums)

In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_nums)

In [ ]:
%%time
enc = dd_trf.fit(dd_nums)

In [ ]:
%%time
enc = dd_trf.transform(dd_nums)

In [ ]:
# TIMING MULTICLASS TARGET ENCODER

In [ ]:
task = task = Task('multiclass')
adv_roles = True
parts = 1
reader = PandasToPandasReader(task, advanced_roles=adv_roles,
                              n_jobs=1)
gpu_reader = CudfReader(task, advanced_roles=adv_roles, n_jobs=1)
dd_reader = DaskCudfReader(task, advanced_roles=adv_roles, 
                           n_jobs=1, compute=False, npartitions=parts)

hy_reader = HybridReader(task, num_cpu_readers=1,
                         num_gpu_readers=2, gpu_ratio=0.6,
                         output='mgpu', advanced_roles=adv_roles,
                         npartitions=parts, n_jobs=1)

In [ ]:
data_len = data.shape[0]
data['TARGET'] = pd.Series(np.random.randint(0, 4, data_len)).astype('i')

In [ ]:
gpu_data = cudf.DataFrame.from_pandas(data, nan_as_null=False)
dd_data = dask_cudf.from_cudf(gpu_data, npartitions=parts)

In [ ]:
%%time
ds = reader.fit_read(data, roles = {'target': 'TARGET'})

In [ ]:
%%time
gpu_ds = gpu_reader.fit_read(gpu_data, roles = {'target': 'TARGET'})

In [ ]:
%%time
dd_ds = dd_reader.fit_read(dd_data, roles = {'target': 'TARGET'})

In [ ]:
cats = ds[:, get_columns_by_role(ds, 'Category')]
gpu_cats = gpu_ds[:, get_columns_by_role(gpu_ds, 'Category')]
dd_cats = dd_ds[:, get_columns_by_role(dd_ds, 'Category')]

#cpu cats have zero cat columns
print(cats.shape, gpu_cats.shape, dd_cats.shape)

In [ ]:
trf = SequentialTransformer(
    [categorical.LabelEncoder(), categorical.MultiClassTargetEncoder()]
)
gpu_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.MultiClassTargetEncoder_gpu()]
)
dd_trf = SequentialTransformer(
    [categorical_gpu.LabelEncoder_gpu(), categorical_gpu.MultiClassTargetEncoder_gpu()]
)


In [ ]:
%%timeit
enc = gpu_trf.fit_transform(gpu_cats)

In [ ]:
%%timeit
enc = dd_trf.fit_transform(dd_cats)